In [38]:
# Import relevant functionality
# from langchain_deepseek import ChatDeepSeek
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv

# 加载.env文件中的环境变量
load_dotenv()

def get_weather(city: str) -> str:  # (1)!
    """Get weather for a given city."""
    # 用字典存储城市对应的天气
    weather_dict = {
        "杭州": "晴天",
        "上海": "多云",
        "北京": "小雨"
    }
    # 通过get方法查询，若城市不在字典中则返回"未知"
    weather = weather_dict.get(city, "未知")
    # 若查询到具体天气则格式化输出，否则直接返回"未知"
    return f"{city}现在是{weather}!" if weather != "未知" else weather

In [39]:
# model = ChatDeepSeek(
#         model="deepseek-chat",
#         temperature=0,
#         max_tokens=1024,
#         timeout=None,
#         max_retries=2)

from langchain_openai import ChatOpenAI

model = ChatOpenAI(base_url="http://192.168.88.15:1025/v1/", model="qwen3-4b")

response = model.invoke([HumanMessage(content="你好")])
response.content

'你好！有什么我可以帮你的吗？😊'

In [40]:
tools = [get_weather]

model_with_tools = model.bind_tools(tools)

In [41]:
response = model_with_tools.invoke([HumanMessage(content="你好")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 你好！有什么可以帮助你的吗？
ToolCalls: []


In [42]:
response = model_with_tools.invoke([HumanMessage(content="杭州现在天气怎么样")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'get_weather', 'args': {'city': '杭州'}, 'id': 'call_OzkpgQwm', 'type': 'tool_call'}]


### 创建智能体

In [43]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

### 运行智能体

In [44]:
response = agent_executor.invoke({"messages": [HumanMessage(content="你好")]})

response["messages"][-1].content

'你好！有什么可以帮助你的吗？'

In [50]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="杭州现在天气怎么样")]}
)
response["messages"][-1].content

'杭州现在是晴天!'

### 流式消息

In [51]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="杭州现在天气怎么样？")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bCG065HF', 'function': {'arguments': '{"city": "杭州"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 155, 'total_tokens': 175, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'batch_size': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'queue_wait_time': [5153, 55, 96, 47, 44, 39, 53, 38, 47, 38, 27, 22, 23, 24, 29, 35, 26, 28, 41, 48]}, 'model_name': 'qwen3-4b', 'system_fingerprint': None, 'id': 'endpoint_common_48167', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9343d6d8-4fb2-4bd7-b36b-8cf21ba63341-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '杭州'}, 'id': 'call_bCG065HF', 'type': 'tool_call'}], usage_metadata={'input_tokens': 155, 'output_tokens': 20, 'total_tokens': 175, 'input_token_details': {}, 'output_token_detail

### 流式Token

In [52]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="杭州现在天气怎么样？")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: get_weather with inputs: {'city': '杭州'}
Done tool: get_weather
Tool output was: content='杭州现在是晴天!' name='get_weather' tool_call_id='call_QodUI5cR'
--
杭州|现在|是|晴|天|!|

### 添加记忆

In [53]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="你好，我是小明")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='你好，小明！有什么我可以帮你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 155, 'total_tokens': 167, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'batch_size': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'queue_wait_time': [5159, 49, 45, 37, 32, 28, 37, 30, 32, 40, 32, 33]}, 'model_name': 'qwen3-4b', 'system_fingerprint': None, 'id': 'endpoint_common_48171', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--fe36da4a-ed10-41e7-801a-5e7b1ad5c9ce-0', usage_metadata={'input_tokens': 155, 'output_tokens': 12, 'total_tokens': 167, 'input_token_details': {}, 'output_token_details': {}})]}}
----


In [54]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="我的名字叫什么?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='你的名字是小明。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 181, 'total_tokens': 188, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'batch_size': [1, 1, 1, 1, 1, 1, 1], 'queue_wait_time': [5215, 40, 32, 39, 36, 28, 30]}, 'model_name': 'qwen3-4b', 'system_fingerprint': None, 'id': 'endpoint_common_48172', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--c7424928-3378-42e4-846f-858f9a4d5a5b-0', usage_metadata={'input_tokens': 181, 'output_tokens': 7, 'total_tokens': 188, 'input_token_details': {}, 'output_token_details': {}})]}}
----


In [55]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="我的名字叫什么？")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='我无法知道您的名字。您可以告诉我您的名字吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 155, 'total_tokens': 168, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'batch_size': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'queue_wait_time': [5199, 55, 49, 37, 32, 34, 32, 30, 32, 31, 34, 35, 49]}, 'model_name': 'qwen3-4b', 'system_fingerprint': None, 'id': 'endpoint_common_48173', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--04d644d1-62fa-475d-872b-fe540c69ad28-0', usage_metadata={'input_tokens': 155, 'output_tokens': 13, 'total_tokens': 168, 'input_token_details': {}, 'output_token_details': {}})]}}
----
